# CS4035 - Cyber Data Analytics
## Lab 4

### Submit on brightspace (zip file with the name Group_xx.zip)
This jupyter file completed with code, plots, figures and report for each question. Write the code or explanation below each sub question. For the explanations, include what you would normally include in the report for this lab assignment, for example data pre-processing, hypothesis tested, approach, results, etc.

(if you used hard-to-get libraries) The libraries needed to run this file. 

Your peers should be able to use the readme section for instructions and be able to run this file. 

## Group Number : 42

## Student 1 
### Name : Otte Van Dam
### ID : 5096790

## Student 2
### Name : Suhaib Basir
### ID : 5059151

## README

Provide instructions - libraries used, location of the data file, etc. Keep it short. Remember your peers will not debug your code and should be able to reproduce the exact output you provide.

## 1. Preparation (0 points)
### Load imports

In [2]:
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt

from tqdm import tqdm

### 1a. Load the dataset and split it into a train and test set

In [3]:
X = np.load('X.npy')
y = np.load('y.npy')

# Split the data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
model = SGDClassifier(loss='log_loss', max_iter=1000, tol=1e-3)

# Fit the model to the training set
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy before attack:", accuracy)

Accuracy before attack: 0.9179276315789474


## 2. Attacking linear models (5 Points)

### 2a. Implementing useless import attacks


In [5]:
def evade(sample, coef):
    for i, c in enumerate(coef):
        if c < 0:
            sample[i] = 1
    return sample

# Get the coefficients
coefficients = model.coef_

# Create new X which will be for the attack
X_test_attack = []
for sample in X_test:
    X_test_attack.append(evade(sample, coefficients[0]))

# Predict on the test set
y_pred_attack = model.predict(X_test_attack)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred_attack)
print("Accuracy after attack:", accuracy)

Accuracy after attack: 0.5024671052631579


### 2b. Analysis and answers to the questions
The accuracy drops from 92% to 50%. This shows that the model is not robust at all. At this point a random classifier is just as good.

In [11]:
# Define batch size and calculate the number of batches
batch_size = 100
num_batches = int(np.ceil(len(X_train) / batch_size))

# Create an instance of the SGDClassifier model
model_hardened = SGDClassifier(loss='log_loss')

coefficients_temporary = None

# Train the model on the training set in batches
for batch in tqdm(range(num_batches)):
    start_idx = batch * batch_size
    end_idx = (batch + 1) * batch_size
    X_batch = X_train[start_idx:end_idx]
    X_batch_attack = []
    if coefficients_temporary is not None:
        # Create new X which will be for the attack
        for sample in X_batch:
            X_batch_attack.append(evade(sample, coefficients_temporary[0]))
    else:
        X_batch_attack = X_batch
    y_batch = y_train[start_idx:end_idx]

    model_hardened.partial_fit(X_batch_attack, y_batch, classes=np.unique(y_train))
    coefficients_temporary = model_hardened.coef_

coefficients_hardened = model_hardened.coef_
# Create new X which will be for the attack
X_test_attack_hardened = []
for sample in X_test:
    X_test_attack_hardened.append(evade(sample, coefficients_hardened[0]))


# Predict on the test set
y_pred_attack_hardened = model_hardened.predict(X_test_attack_hardened)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred_attack_hardened)
print("Accuracy after attack with hardening:", accuracy)

100%|██████████| 244/244 [01:04<00:00,  3.81it/s]


Accuracy after attack with hardening: 0.4975328947368421


## 3. Improving robustness by hardening (5 Points)

### 3a. Implementing hardening

In [7]:
accuracy_list_hardened = []
batch_sizes = [50, 64, 75]
# Define batch size and calculate the number of batches
for batch_size in tqdm(batch_sizes):
    num_batches = int(np.ceil(len(X_train) / batch_size))

    # Create an instance of the SGDClassifier model
    model_hardened = SGDClassifier(loss='log_loss', max_iter=1000, tol=1e-3)

    coefficients_temporary = coefficients

    # Train the model on the training set in batches
    for batch in range(num_batches):
        start_idx = batch * batch_size
        end_idx = (batch + 1) * batch_size
        X_batch = X_train[start_idx:end_idx]
        X_batch_attack = []
        # Create new X which will be for the attack
        for sample in X_batch:
            X_batch_attack.append(evade(sample, coefficients_temporary[0]))
        y_batch = y_train[start_idx:end_idx]

        model_hardened.partial_fit(X_batch_attack, y_batch, classes=np.unique(y_train))
        coefficients_temporary = model_hardened.coef_

    coefficients_hardened = model_hardened.coef_
    # Create new X which will be for the attack
    X_test_attack_hardened = []
    for sample in X_test:
        X_test_attack_hardened.append(evade(sample, coefficients_hardened[0]))


    # Predict on the test set
    y_pred_attack_hardened = model_hardened.predict(X_test_attack_hardened)

    # Calculate the accuracy
    accuracy = accuracy_score(y_test, y_pred_attack_hardened)
    accuracy_list_hardened.append(accuracy)

 33%|███▎      | 1/3 [01:52<03:44, 112.41s/it]


KeyboardInterrupt: 

### 3b. Analysis and answers to the questions

In [ ]:
print(accuracy_list_hardened)

## 4. Training non-negative linear models (5 Points)

### 4a. Implement non-negative logistic regression using SGD

In [ ]:
batch_sizes = [4, 8, 20, 35, 50, 60, 70]
accuracy_list_non_negative = []
for batch_size in tqdm(batch_sizes):
    # Define batch size and calculate the number of batches
    num_batches = int(np.ceil(len(X_train) / batch_size))

    # Create an instance of the SGDClassifier model
    model_non_negative = SGDClassifier(loss='log_loss', max_iter=1000, tol=1e-3)

    # Train the model on the training set in batches
    for batch in range(num_batches):
        start_idx = batch * batch_size
        end_idx = (batch + 1) * batch_size
        X_batch = X_train[start_idx:end_idx]
        y_batch = y_train[start_idx:end_idx]

        model_non_negative.partial_fit(X_batch, y_batch, classes=np.unique(y_train))

        # Retrieve the coefficients
        coefficients = model_non_negative.coef_

        # Set any negative coefficients to 0
        coefficients[coefficients < 0] = 0

        # Set the modified coefficients back to the model
        model_non_negative.coef_ = coefficients

    coefficients_non_negative = model_non_negative.coef_
    # Create new X which will be for the attack
    X_test_attack_non_negative = []
    for sample in X_test:
        X_test_attack_non_negative.append(evade(sample, coefficients_non_negative[0]))


    # Predict on the test set
    y_pred_attack_non_negative = model_non_negative.predict(X_test_attack_non_negative)

    # Calculate the accuracy
    accuracy = accuracy_score(y_test, y_pred_attack_non_negative)
    accuracy_list_non_negative.append(accuracy)

In [ ]:
print(model_non_negative.t_)

### 4b. Analysis and answers to the questions.

In [ ]:
# Plot the accuracies for each batch size
plt.plot(batch_sizes, accuracy_list_hardened, marker='o', label='Hardened Model')
plt.plot(batch_sizes, accuracy_list_non_negative, marker='o', label='Non-Negative Model')

# Add labels and title to the plot
plt.xlabel('Batch Size')
plt.ylabel('Accuracy')
plt.title('Accuracy vs. Batch Size')

# Add legend
plt.legend()

# Show the plot
plt.show()

In [ ]:
print(accuracy_list_non_negative)
print(accuracy_list_hardened)
print(batch_sizes)

## 5. Bonus: robust decision trees for fraud detection (5 Points)

### 5a. Train and analyze a regular decision tree for fraud detection

### 5b. Train and analyze a robust decision tree (GROOT) for fraud detection

### 5c. Evaluate the robustness of the two models

In [ ]:
# Define batch size and calculate the number of batches
batch_size = 64
num_batches = int(np.ceil(len(X_train) / batch_size))

# Create an instance of the SGDClassifier model
model_hardened = SGDClassifier(loss='log_loss', max_iter=1000, tol=1e-3)

coefficients_temporary = [[0] * len(X_train[0])]

# Train the model on the training set in batches
for batch in tqdm(range(num_batches)):
    start_idx = batch * batch_size
    end_idx = (batch + 1) * batch_size
    X_batch = X_train[start_idx:end_idx]
    X_batch_attack = []
    # Create new X which will be for the attack
    for sample in X_batch:
        X_batch_attack.append(evade(sample, coefficients_temporary[0]))
    y_batch = y_train[start_idx:end_idx]

    model_hardened.partial_fit(X_batch_attack, y_batch, classes=np.unique(y_train))
    coefficients_temporary = model_hardened.coef_

coefficients_hardened = model_hardened.coef_
# Create new X which will be for the attack
X_test_attack_hardened = []
for sample in X_test:
    X_test_attack_hardened.append(evade(sample, coefficients_hardened[0]))


# Predict on the test set
y_pred_attack_hardened = model_hardened.predict(X_test_attack_hardened)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred_attack_hardened)
print("Accuracy after attack with hardening:", accuracy)

In [ ]:
batch_size = 4
# Define batch size and calculate the number of batches
num_batches = int(np.ceil(len(X_train) / batch_size))

# Create an instance of the SGDClassifier model
model_non_negative = SGDClassifier(loss='log_loss', max_iter=1000, tol=1e-3, shuffle=False)

# Train the model on the training set in batches
for batch in range(num_batches):
    start_idx = batch * batch_size
    end_idx = (batch + 1) * batch_size
    X_batch = X_train[start_idx:end_idx]
    y_batch = y_train[start_idx:end_idx]

    model_non_negative.partial_fit(X_batch, y_batch, classes=np.unique(y_train))

    # Retrieve the coefficients
    coefficients = model_non_negative.coef_

    # Set any negative coefficients to 0
    coefficients[coefficients < 0] = 0

    # Set the modified coefficients back to the model
    model_non_negative.coef_ = coefficients

coefficients_non_negative = model_non_negative.coef_
# Create new X which will be for the attack
X_test_attack_non_negative = []
for sample in X_test:
    X_test_attack_non_negative.append(evade(sample, coefficients_non_negative[0]))


# Predict on the test set
y_pred_attack_non_negative = model_non_negative.predict(X_test_attack_non_negative)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred_attack_non_negative)
print(accuracy)